In [1]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/annex1', './input/annex2']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [2]:
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
import numpy as np
# ALEX: remove plotting
# import matplotlib.pyplot as plt
# import seaborn as sns

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
df = pd.read_csv('./input/annex1.scaled.csv')
df.head()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 22:06:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/26 22:06:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/26 22:06:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,Item Code,Item Name,Category Code,Category Name
0,102900005115168,Niushou Shengcai,1011010101,Flower/Leaf Vegetables
1,102900005115199,Sichuan Red Cedar,1011010101,Flower/Leaf Vegetables
2,102900005115625,Local Xiaomao Cabbage,1011010101,Flower/Leaf Vegetables
3,102900005115748,White Caitai,1011010101,Flower/Leaf Vegetables
4,102900005115762,Amaranth,1011010101,Flower/Leaf Vegetables


In [4]:
df1 = pd.read_csv('./input/annex2.scaled.csv')
df1.head()

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (RMB/kg),Sale or Return,Discount (Yes/No)
0,2020-07-01,2025-01-26 09:15:07.924,102900005117056,0.396,7.6,sale,No
1,2020-07-01,2025-01-26 09:17:27.295,102900005115960,0.849,3.2,sale,No
2,2020-07-01,2025-01-26 09:17:33.905,102900005117056,0.409,7.6,sale,No
3,2020-07-01,2025-01-26 09:19:45.450,102900005115823,0.421,10.0,sale,No
4,2020-07-01,2025-01-26 09:20:23.686,102900005115908,0.539,8.0,sale,No


In [5]:
# Join the both DataFrames

merge = pd.merge(df, df1, on ='Item Code')
merge.head()

,Item Code,Item Name,Category Code,Category Name,Date,Time,Quantity Sold (kilo),Unit Selling Price (RMB/kg),Sale or Return,Discount (Yes/No)
0,102900005117056,Paopaojiao (Jingpin),1011010504,Capsicum,2020-07-01,2025-01-26 09:15:07.924,0.396,7.6,sale,No
1,102900005115960,Chinese Cabbage,1011010101,Flower/Leaf Vegetables,2020-07-01,2025-01-26 09:17:27.295,0.849,3.2,sale,No
2,102900005117056,Paopaojiao (Jingpin),1011010504,Capsicum,2020-07-01,2025-01-26 09:17:33.905,0.409,7.6,sale,No
3,102900005115823,Shanghaiqing,1011010101,Flower/Leaf Vegetables,2020-07-01,2025-01-26 09:19:45.450,0.421,10.0,sale,No
4,102900005115908,Caixin,1011010101,Flower/Leaf Vegetables,2020-07-01,2025-01-26 09:20:23.686,0.539,8.0,sale,No


# Data Preparation

In [6]:
# Rename Columns
merge =merge.rename(columns={'Quantity Sold (kilo)':'Quantity Sold', 'Unit Selling Price (RMB/kg)':'Unit Selling Price','Discount (Yes/No)':'Discount'})

In [7]:
#Change datatype of date column.

merge['Date']=pd.to_datetime(merge['Date'])

In [8]:
evaluate_lazy(merge.isnull().sum())

25/01/26 22:06:53 WARN AttachDistributedSequenceExec: clean up cached RDD(52) in AttachDistributedSequenceExec(333)


Item Code             0
Item Name             0
Category Code         0
Category Name         0
Date                  0
Time                  0
Quantity Sold         0
Unit Selling Price    0
Sale or Return        0
Discount              0
dtype: int64

# Exploratory Data Analysis

In [9]:
Category_count=merge['Category Name'].value_counts()
evaluate_lazy(Category_count)

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(


Flower/Leaf Vegetables         331968
Capsicum                       207996
Edible Mushroom                148424
Cabbage                         86570
Aquatic Tuberous Vegetables     58647
Solanum                         44898
Name: Category Name, dtype: int64

In [10]:
# ALEX: remove plotting
# plt.figure(figsize=(8,6))
# colors = ['blue', 'green', 'red', 'purple', 'orange', 'yellow']
# plt.bar(Category_count.index, Category_count.values, color = colors)
# plt.title("Category wise Order Count")
# plt.xticks(rotation=90)
# plt.show()
Category_count.index
Category_count.values

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([331968, 207996, 148424,  86570,  58647,  44898])

In [11]:
top_10_products =merge['Item Name'].value_counts()[:10]
evaluate_lazy(top_10_products)

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(


Wuhu Green Pepper (1)    69945
Broccoli                 58906
Xixia Mushroom (1)       47509
Yunnan Shengcai          39887
Net Lotus Root (1)       39285
Eggplant (2)             27455
Paopaojiao (Jingpin)     25338
Yunnan Lettuces          23265
Luosi Pepper             22678
Chinese Cabbage          19148
Name: Item Name, dtype: int64

In [12]:
# ALEX: remove plotting
# plt.figure(figsize=(10,6))
# colors = ['red', 'green', 'blue', 'yellow', 'purple', 'orange', 'pink', 'teal', 'brown', 'gray']

# plt.bar(top_10_products.index, top_10_products.values, color = colors)
# plt.title("Top 10 Products by its Order count")
# plt.xticks(rotation=90)
# plt.show()
top_10_products.index
top_10_products.values

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([69945, 58906, 47509, 39887, 39285, 27455, 25338, 23265, 22678,
       19148])

In [13]:
# Creating sales column

merge['Sales']= merge['Unit Selling Price'] * merge['Quantity Sold']

In [14]:
Item_sales = merge.groupby('Item Name',as_index=False)['Sales'].sum().sort_values(by='Sales',ascending=False).head(10)
evaluate_lazy(Item_sales)

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(
/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,Item Name,Sales
8,Broccoli,269880.9572
58,Net Lotus Root (1),211652.1013
17,Xixia Mushroom (1),211198.3864
15,Wuhu Green Pepper (1),205113.6991
26,Yunnan Shengcai,129756.9615
42,Eggplant (2),117729.4239
39,Paopaojiao (Jingpin),95568.6339
4,Luosi Pepper,82009.3451
20,Yunnan Lettuces,70665.1061
61,Chinese Cabbage,65675.4366


In [15]:
# ALEX: remove plotting
# plt.figure(figsize=(10,8))
# ab=sns.barplot(x='Item Name',y='Sales',data=Item_sales)
# ab.bar_label(ab.containers[0])
# plt.title('Top 10 Products by its Sales')
# plt.xlabel('Product Name')
# plt.ylabel('Sales')
# plt.xticks(rotation=90)
# plt.show()

In [16]:
#Add year column in dataframe
merge['Year'] = merge['Date'].dt.year

In [17]:
Year_wise_sales=merge.groupby('Year')['Sales'].sum()
evaluate_lazy(Year_wise_sales)

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(
25/01/26 22:07:02 WARN AttachDistributedSequenceExec: clean up cached RDD(157) in AttachDistributedSequenceExec(1608)


Year
2020    6.695293e+05
2021    1.100363e+06
2022    1.036772e+06
2023    5.631021e+05
Name: Sales, dtype: float64

In [18]:
# ALEX: remove plotting
# plt.figure(figsize=(6,6))
# plt.pie(Year_wise_sales,autopct ='%.2f%%',labels =Year_wise_sales.index)
# plt.title('Year Wise Sales')
# plt.show()
evaluate_lazy(Year_wise_sales.index)

Int64Index([2020, 2021, 2022, 2023], dtype='int64', name='Year')

In [19]:
Sale_Or_Return =merge['Sale or Return'].value_counts()
Sale_Or_Return= pd.DataFrame(Sale_Or_Return)
evaluate_lazy(Sale_Or_Return)

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,Sale or Return
sale,878042
return,461


In [20]:
Discount =merge['Discount'].value_counts()
Discount = pd.DataFrame(Discount)
evaluate_lazy(Discount)

/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/base.py:1437: FutureWarning: The resulting Series will have a fixed name of 'count' from 4.0.0.
  warnings.warn(
/home/alex/Programs/python/dias/.venv/lib64/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,Discount
No,831137
Yes,47366


# Conclusion :
#### 1. Most of the people prefer to purchase flower/leaf vegetables from supermarket.
#### 2. Top 10 Products are : Wuhu Green Pepper, Broccoli                , Xixia Mushroom (1)      , Yunnan Shengcai          , Net Lotus Root (1)       , Eggplant (2)            , Paopaojiao (Jingpin)    , Yunnan Lettuces          , Luosi Pepper             , Chinese Cabbage. 
#### 3. Top 10 products by Sales Generation are : Broccoli, , Net Lotus Root (1), Xixia Mushroom (1), Wuhu Green Pepper (1), Yunnan Shengcai, Eggplant (2), Paopaojiao (Jingpin), Luosi Pepper, Yunnan Lettuces, Chinese Cabbage.
#### 4. The year 2021 has higest sales around 32%.
#### 5. From 9 lakh products only 461 products are return.
#### 6. Out of 900,000 orders, only 47,000 received discounts.

# Model Training

In [21]:
merge.head()

,Item Code,Item Name,Category Code,Category Name,Date,Time,Quantity Sold,Unit Selling Price,Sale or Return,Discount,Sales,Year
0,102900005117056,Paopaojiao (Jingpin),1011010504,Capsicum,2020-07-01,2025-01-26 09:15:07.924,0.396,7.6,sale,No,3.0096,2020
1,102900005115960,Chinese Cabbage,1011010101,Flower/Leaf Vegetables,2020-07-01,2025-01-26 09:17:27.295,0.849,3.2,sale,No,2.7168,2020
2,102900005117056,Paopaojiao (Jingpin),1011010504,Capsicum,2020-07-01,2025-01-26 09:17:33.905,0.409,7.6,sale,No,3.1084,2020
3,102900005115823,Shanghaiqing,1011010101,Flower/Leaf Vegetables,2020-07-01,2025-01-26 09:19:45.450,0.421,10.0,sale,No,4.2100,2020
4,102900005115908,Caixin,1011010101,Flower/Leaf Vegetables,2020-07-01,2025-01-26 09:20:23.686,0.539,8.0,sale,No,4.3120,2020


In [22]:
merge = merge.drop(['Date'],axis=1)

In [23]:
# ALEX: remove ML code
# from sklearn.preprocessing import LabelEncoder
# le =LabelEncoder()

In [24]:
#Store all categorical columns
categorical_columns = merge.select_dtypes(include=['object']).columns

In [25]:
# Converting all categorical columns into numeric
for col in categorical_columns:
# ALEX: remove ML code
#     merge[col] = le.fit_transform(merge[col])
    merge[col] = merge[col]
    
merge.head()

,Item Code,Item Name,Category Code,Category Name,Time,Quantity Sold,Unit Selling Price,Sale or Return,Discount,Sales,Year
0,102900005117056,Paopaojiao (Jingpin),1011010504,Capsicum,2025-01-26 09:15:07.924,0.396,7.6,sale,No,3.0096,2020
1,102900005115960,Chinese Cabbage,1011010101,Flower/Leaf Vegetables,2025-01-26 09:17:27.295,0.849,3.2,sale,No,2.7168,2020
2,102900005117056,Paopaojiao (Jingpin),1011010504,Capsicum,2025-01-26 09:17:33.905,0.409,7.6,sale,No,3.1084,2020
3,102900005115823,Shanghaiqing,1011010101,Flower/Leaf Vegetables,2025-01-26 09:19:45.450,0.421,10.0,sale,No,4.2100,2020
4,102900005115908,Caixin,1011010101,Flower/Leaf Vegetables,2025-01-26 09:20:23.686,0.539,8.0,sale,No,4.3120,2020


In [26]:
# Define x variables (independents variable)
x= merge.drop(['Sale or Return'],axis=1)

In [27]:
# Define y variable (dependent variable)
y = merge['Sale or Return']

In [28]:
# ALEX: remove ML code
# from sklearn.model_selection import train_test_split

In [29]:
# Split data into train and test datasets
# ALEX: remove ML code
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [30]:
# ALEX: remove ML code
# len(x_train)

In [31]:
# ALEX: remove ML code
# len(x_test)

In [32]:
# ALEX: remove ML code
# from sklearn.ensemble import RandomForestClassifier
# model =RandomForestClassifier()

In [33]:
# ALEX: remove ML code
# model.fit(x_train,y_train)

In [34]:
# ALEX: remove ML code
# model.predict(x_test)

In [35]:
# ALEX: remove ML code
# from sklearn.model_selection import cross_val_score

In [36]:
# ALEX: remove ML code
# score =cross_val_score(RandomForestClassifier(),x,y)
# score

In [37]:
# ALEX: remove ML code
# score.mean()

#### Our model score is almost 99%.